In [76]:
# [Deteccion de ondas Gravitacionales]
# Autor: Felipe de Jesus Felix Arredondo 
# Fecha: 2024/04/03

# [Libraries] Importamos las librerias necesarias
# [Library](Gravitational Waves Data){GWData}
from GWData.GWGenerator import GWDataset

# [Library](Preprocessing){GWPreprocessing}
from GWData.Preprocessing import GWPreprocessing

# [Library](Persistencia){gtda}
from gtda.diagrams import PersistenceEntropy

# [Library](Matriz de datos){numpy}
import numpy as np

# [Library](Graficas){matplotlib}
import matplotlib.pyplot as plt
from gtda.plotting import plot_point_cloud
import plotly.express as px

# [Library](Data management){sklearn}
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

# a mi no me gustan los warnings
import warnings
import time
warnings.filterwarnings("ignore")

# [Library](File system){os}
import os
import pathlib

# [Library](Deep learning){Tensorflow}
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Conv2D

# [Library](Deep learning){Pytorch}
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# [Mini-class]{Clas}
def Classifier():
    # [Model]{Sequential}
    Orthogonal = keras.initializers.Orthogonal()
    model = keras.Sequential([
        Conv2D(32, kernel_size=16, activation='relu', input_shape=(32,1671, 3), kernel_initializer=Orthogonal),
        MaxPooling1D(4),
        #Dense(64, activation='relu'),
        Conv1D(64, kernel_size=16,activation='relu', kernel_initializer=Orthogonal),
        MaxPooling1D(4),
        #Dense(128, activation='relu'),
        Conv1D(128, kernel_size=16, activation='relu', kernel_initializer=Orthogonal),
        MaxPooling1D(4),
        #Dense(256, activation='relu'),
        Conv1D(256, kernel_size=16, activation='relu', kernel_initializer=Orthogonal),
        MaxPooling1D(4),
        #Dense(512, activation='relu'),
        Flatten(),
        Dense(128, activation='relu', kernel_initializer=Orthogonal),
        Dense(64, activation='relu', kernel_initializer=Orthogonal),
        Dense(2, activation='softmax')
    ])
    # [Model]{Compile}
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# [Main]
if __name__ == '__main__':
    # [Data generator]{Generamos los datos}
    generator = GWDataset(n_signals=1000)

    Preprocessing = GWPreprocessing( # Define the Preprocessing object
        dimension=200, # Embedding dimension
        delay=10, # Tamaño de la ventana
        stride=10,  # Stride o espacio entre las ventanas
        components=3 # Numero de componentes principales
    )
    # [Data]{Obtenemos los datos}
    print("Obteniendo los datos")
    (RAW,Y) = generator.getTrainData() # Obtenemos los datos

    # [Data]{Preprocesamos los datos}
    print("Preprocesando los datos")
    startT = time.time()
    # Estandarizamos los datos
    RAW = (RAW - np.mean(RAW, axis=0)) / np.std(RAW, axis=0)
    (DATA, HOMOLOGY) = Preprocessing.fit(RAW) # Preprocesamos los datos
    # Standarizamos los datos
    print ("Tiempo de preprocesamiento: ", time.time()-startT)
    print ("Dimensiones de los datos: ", DATA.shape)
    print ("Dimensiones de la homologia: ", HOMOLOGY.shape)

    # [Data]{Guardamos los datos}
    np.save("Save/DATA.npy", DATA)
    np.save("Save/HOMOLOGY.npy", HOMOLOGY)

Obteniendo los datos
Preprocesando los datos
Tiempo de preprocesamiento:  33.82599115371704
Dimensiones de los datos:  (1000, 671, 3)
Dimensiones de la homologia:  (1000, 1036, 3)


In [77]:
# Visualize a preprocessing example
plot_point_cloud(DATA[0])

In [86]:
# Visualize a persistence diagram
randomSample = np.random.randint(0, HOMOLOGY.shape[0])
fig = px.scatter(x=HOMOLOGY[randomSample][:, 0], y=HOMOLOGY[randomSample][:, 1])
fig.show()
# Imprime la clase
print(Y[randomSample])

[1.]


In [79]:
from gtda.plotting import plot_diagram

plot_diagram(HOMOLOGY[randomSample])

In [80]:
# Visualize a preprocessing example
plot_point_cloud(DATA[0])

In [81]:
# Import Conv2D
from keras.layers import Conv2D

In [84]:
# [Mini-class]{Clas}
def Classifier():
    # [Model]{Sequential}
    Orthogonal = keras.initializers.Orthogonal()
    model = keras.Sequential([
        Conv1D(64, kernel_size=16, activation='relu', input_shape=(1707, 3), kernel_initializer=Orthogonal, name='conv1'),
        MaxPooling1D(strides=4, pool_size=4),
        Dense(64, activation='relu'),
        Conv1D(128, kernel_size=16,activation='relu', kernel_initializer=Orthogonal, name='conv2'),
        MaxPooling1D(strides=4, pool_size=4),
        Dense(128, activation='relu'),
        Conv1D(256, kernel_size=16, activation='relu', kernel_initializer=Orthogonal, name='conv3'),
        MaxPooling1D(strides=4, pool_size=4),
        Dense(256, activation='relu'),
        Conv1D(512, kernel_size=16, activation='relu', kernel_initializer=Orthogonal, name='conv4'),
        MaxPooling1D(strides=4, pool_size=4),
        Dense(512, activation='relu'),
        Flatten(),
        Dense(128, activation='linear', kernel_initializer=Orthogonal),
        Dense(128, activation='relu', kernel_initializer=Orthogonal),
        Dense(64, activation='linear', kernel_initializer=Orthogonal),
        Dense(64, activation='relu', kernel_initializer=Orthogonal),
        Dense(2, activation='softmax')
    ])
    # [Model]{Compile}
    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=['accuracy']
    )
    return model

In [ ]:
# Version en pytorch
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 16)
        self.pool = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(64, 128, 16)
        self.conv3 = nn.Conv1d(128, 256, 16)
        self.conv4 = nn.Conv1d(256, 512, 16)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 64)

In [85]:
model = Classifier()
# Train the model
# Concatenamos los datos
Final = np.concatenate((DATA, HOMOLOGY), axis=1)
# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(Final, Y, test_size=0.2, random_state=42)
# Entrenamos el modelo
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
25/25 [==============================] - 7s 163ms/step - loss: 0.2538 - accuracy: 0.4938 - val_loss: 0.2500 - val_accuracy: 0.5700
Epoch 2/5
25/25 [==============================] - 3s 106ms/step - loss: 0.2500 - accuracy: 0.5175 - val_loss: 0.2500 - val_accuracy: 0.5700
Epoch 3/5
25/25 [==============================] - 2s 95ms/step - loss: 0.2500 - accuracy: 0.4888 - val_loss: 0.2500 - val_accuracy: 0.5750
Epoch 4/5
25/25 [==============================] - 3s 102ms/step - loss: 0.2500 - accuracy: 0.5063 - val_loss: 0.2500 - val_accuracy: 0.4950
Epoch 5/5
25/25 [==============================] - 3s 108ms/step - loss: 0.2500 - accuracy: 0.4963 - val_loss: 0.2500 - val_accuracy: 0.5650
